In [1]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProject").getOrCreate()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

# Create the list of struct fields
schema = [StructField("customer_id", IngeterType(), False), StructField("price", IntegerType(), True),]
schema

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

In [23]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame 
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("sample_us.tsv"), inferSchema=True, sep='\t', timestampFormat="yyyy-MM-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-08-31 00:00:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

## How many rows are in the dataset?

In [36]:
# Count the number of entries in the dataset
print(f"There are {df.count()} rows in the dataset")

There are 49 rows in the dataset


## Transform DataFrame to fit review_id table

In [43]:
review_df = df.select(["review_id", "customer_id","product_id", "product_parent", "review_date"])
review_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789|2015-08-31 00:00:00|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646|2015-08-31 00:00:00|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353|2015-08-31 00:00:00|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918|2015-08-31 00:00:00|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439|2015-08-31 00:00:00|
|R3BPETL222LMIM|   13394189|B009B7F6CA|     873028700|2015-08-31 00:00:00|
|R3SORMPJZO3F2J|    2749569|B0101EHRSM|     723424342|2015-08-31 00:00:00|
|R2RDOJQ0WBZCF6|   41137196|B00407S11Y|     383363775|2015-08-31 00:00:00|
|R2B8VBEPB4YEZ7|     433677|B00FGPU7U2|     780517568|2015-08-31 00:00:00|
|R1CB783I7B0U52|    1297934|B0013OY0S0|     269360126|2015-08-31 00:00:00|
| R2D90RQQ3V8LH|   520062

In [50]:
# Show the schema to confirm the column type
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [45]:
# Drop the hh:mm:ss from the timestamp
review_df = review_df.withColumn("review_date", date_format(review_df['review_date'], 'yyyy-MM-dd'))
review_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789| 2015-08-31|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646| 2015-08-31|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353| 2015-08-31|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918| 2015-08-31|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439| 2015-08-31|
|R3BPETL222LMIM|   13394189|B009B7F6CA|     873028700| 2015-08-31|
|R3SORMPJZO3F2J|    2749569|B0101EHRSM|     723424342| 2015-08-31|
|R2RDOJQ0WBZCF6|   41137196|B00407S11Y|     383363775| 2015-08-31|
|R2B8VBEPB4YEZ7|     433677|B00FGPU7U2|     780517568| 2015-08-31|
|R1CB783I7B0U52|    1297934|B0013OY0S0|     269360126| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [47]:
# Drop any null values
review_df.dropna()

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: string]

In [48]:
# Check to see if the number of rows matches the dataset
print(f"There are {review_df.count()} rows in the data frame")

There are 49 rows in the data frame


## Transform Dataframe to match products table

In [35]:
products_df = df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00EDBY7X8|Monopoly Junior B...|
|B00D7JFOPC|56 Pieces of Wood...|
|B002LHA74O|Super Jumbo Playi...|
|B00ARPLCGY|Barbie Doll and F...|
|B00UZOPOFW|Emazing Lights eL...|
|B009B7F6CA|Melissa & Doug Wa...|
|B0101EHRSM|Big Bang Cosmic P...|
|B00407S11Y|Fun Express Insec...|
|B00FGPU7U2|Fisher-Price Octo...|
|B0013OY0S0|Claw Climber Goli...|
|B00519PJTW|100 Foot Multicol...|
|B001TCY2DO|Pig Jumbo Foil Ba...|
|B00DOQCWF8|Minecraft Animal ...|
|B004C04I4I|Disney Baby: Eeyo...|
|B00NWGEKBY|Team Losi 8IGHT-E...|
|B00000JS5S|Hot Wheels 48- Ca...|
|B00XPWXYDK|ZuZo 2.4GHz 4 CH ...|
|B00VPXX92W|Teenage Mutant Ni...|
|B00YRA3H4U|Franklin Sports M...|
|B009T8BSQY|Alien Frontiers: ...|
+----------+--------------------+
only showing top 20 rows



In [49]:
# Check the columns to match the schema
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [38]:
# Check to see if the number of rows matches the dataset
print(f"There are {products_df.count()} rows in the data frame")

There are 49 rows in the data frame


## Transform the dataframe to match the customers table

In [40]:
customers_df = df.select(["customer_id"])
customers_df.show()

+-----------+
|customer_id|
+-----------+
|   18778586|
|   24769659|
|   44331596|
|   23310293|
|   38745832|
|   13394189|
|    2749569|
|   41137196|
|     433677|
|    1297934|
|   52006292|
|   32071052|
|    7360347|
|   11613707|
|   13545982|
|   43880421|
|    1662075|
|   18461411|
|   27225859|
|   20494593|
+-----------+
only showing top 20 rows



In [41]:
customers_df = customers_df.groupBy('customer_id').agg({"customer_id": "count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   18461411|             1|
|   44331596|             1|
|   20962528|             1|
|   34874898|             1|
|    6762003|             1|
|    1297934|             1|
|   13328687|             1|
|    2749569|             1|
|   12612039|             1|
|   11613707|             1|
|    5543658|             1|
|    7360347|             1|
|   45601416|             1|
|   27225859|             1|
|   32910511|             1|
|   47546726|             1|
|   13545982|             1|
|   12191231|             1|
|     433677|             1|
|   13394189|             1|
+-----------+--------------+
only showing top 20 rows



## Transform dataset to match the vine table

In [42]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDIJS7QYB6XNR|          5|            0|          0|   N|
|R36ED1U38IELG8|          5|            0|          0|   N|
| R1UE3RPRGCOLD|          2|            1|          1|   N|
|R298788GS6I901|          5|            0|          0|   N|
|  RNX4EXOBBPN5|          1|            1|          1|   N|
|R3BPETL222LMIM|          5|            0|          0|   N|
|R3SORMPJZO3F2J|          3|            2|          2|   N|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|
|R1CB783I7B0U52|          1|            0|          1|   N|
| R2D90RQQ3V8LH|          5|            0|          0|   N|
|R1Y4ZOUGFMJ327|          5|            0|          0|   N|
|R2BUV9QJI2A00X|          5|            0|          1|   N|
| RSUHRJFJIRB3Z|          4|            